In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Activation, Dropout, Flatten, BatchNormalization, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import LeakyReLU
from keras.backend import binary_crossentropy


img_width, img_height = 227, 227

train_data_path = '/Users/toobarahimnia/Documents/Software_projects/chest_xray/train'
test_data_path = '/Users/toobarahimnia/Documents/Software_projects/chest_xray/test'
train_sample_size = 5216
test_sample_size = 624
epochs = 3
batch_size = 326
input_shape = (img_width, img_height, 3)

CoroNeuro = Sequential()

#conv layer 1
CoroNeuro.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11,11), strides=(4,4), padding='same'))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('relu'))
CoroNeuro.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#conv layer 2
CoroNeuro.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('relu'))
CoroNeuro.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#conv layer 3
CoroNeuro.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('relu'))
CoroNeuro.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#conv layer 4
CoroNeuro.add(Conv2D(filters=384,  kernel_size=(3,3), strides=(1,1), padding='same'))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('relu'))
CoroNeuro.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#conv layer 5
CoroNeuro.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('relu'))
CoroNeuro.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))


#conv layer to dense/fully connected layer
CoroNeuro.add(Flatten())

#Dense layer 1
CoroNeuro.add(Dense(4096, input_shape=(224, 224, 3)))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('relu'))
CoroNeuro.add(Dropout(0.4))

#Dense layer 2
CoroNeuro.add(Dense(4096))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('relu'))
CoroNeuro.add(Dropout(0.4))

#Dense layer 2
CoroNeuro.add(Dense(1000))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('relu'))
CoroNeuro.add(Dropout(0.4))

#Output layer
CoroNeuro.add(Dense(1))
CoroNeuro.add(BatchNormalization())
CoroNeuro.add(Activation('softmax'))


#Summary
CoroNeuro.summary()

#Compilation
CoroNeuro.compile(loss = binary_crossentropy, optimizer= 'adam', metrics=['accuracy'])


#Augmenting training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.25,
    zoom_range=0.3,
    horizontal_flip=True)

#Augmenting testing data
test_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


#Training

CoroNeuro.fit_generator(
    train_generator,
     
    steps_per_epoch=train_sample_size // batch_size,
    
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_sample_size)

CoroNeuro.save_weights('coro_neuro.h5')

#Evaluate accuracy
score = CoroNeuro.evaluate_generator(test_generator, test_sample_size/batch_size, workers=1)

#Print results
print("Loss: ", score[0], "Accuracy: ", score[1])




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 57, 57, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 57, 57, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 57, 57, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 29, 256)       1024      
_________________________________________________________________
activation_1 (Activation)    (None, 29, 29, 256)       0